# Deep Learning Tutorial 2: Training

In [1]:
from keras.models import Model
from keras.layers import *
from keras.utils.visualize_util import plot


import numpy as np

from callbacks import AUCHistory

Using Theano backend.
Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN not available)


In [2]:
saved = np.load("data.npz")
data = saved["records"]
labels = saved["labels"]

In [3]:
data.shape

(74574, 100, 78)

In [32]:
data[95][30]

array([  4.00078703e+12,  -8.80335995e-01,   4.78135492e+00,
        -2.98849258e-02,  -9.37220281e-03,  -8.13419053e-03,
        -4.40190454e-01,  -8.06269862e-03,  -3.29334036e-02,
        -2.69698235e-02,  -1.22561354e-02,  -1.19421368e+00,
        -2.27359295e-03,  -2.09233030e-03,  -1.23819774e-02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.00000000e+00,

In [13]:
n_drives = data.shape[0]
n_records = data.shape[1]
n_features = data.shape[2]

In [14]:
input = Input(shape=(n_records, n_features), name="inputs")

In [ ]:
x = input

In [15]:
#x = Masking()(input)

In [16]:
x = LSTM(20, return_sequences=True)(x)
x = LSTM(20)(x)

In [17]:
output = Dense(1, activation='sigmoid', name='output')(x)

In [18]:
model = Model(input=input, output=output)

In [19]:
model.compile(optimizer="adam", loss="binary_crossentropy")

In [4]:
model.fit(data, labels, nb_epoch=10, batch_size=100, validation_split=0.2, callbacks=[AUCHistory()])

NameError: name 'model' is not defined